In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score,roc_curve,roc_auc_score,auc, f1_score,recall_score, precision_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from xgboost.sklearn import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.model_selection import KFold

from sklearn.multioutput import MultiOutputClassifier

In [ ]:
np.random.seed(11)

In [ ]:
train = pd.read_csv('../input/lish-moa/train_features.csv')
targets = pd.read_csv('../input/lish-moa/train_targets_scored.csv')

In [ ]:
test = pd.read_csv('../input/lish-moa/test_features.csv')
sub = pd.read_csv('../input/lish-moa/sample_submission.csv')


In [ ]:
train_targets_combined = pd.merge(train,targets)

In [ ]:
train_targets_combined

In [ ]:
new = train_targets_combined[train_targets_combined['cp_type']=='trt_cp']
new

In [ ]:
X_train = new.iloc[:,0:876]
y_train = new.iloc[:,876:]

In [ ]:
X_test = test.drop('sig_id', axis = 1)
X_train.drop('sig_id', axis = 1, inplace = True)

In [ ]:
cp_dose_dict = {'D1':0, 'D2':1}

X_train['cp_dose'] = X_train['cp_dose'].map(cp_dose_dict)
X_test['cp_dose'] = X_test['cp_dose'].map(cp_dose_dict)

In [ ]:
X_train.drop('cp_type', axis = 1, inplace = True)
X_test.drop('cp_type', axis = 1, inplace = True)

In [ ]:
'''def model_selection(X,y):

    np.random.seed(1)

    model_select_df = pd.DataFrame()
    
    model_select_df['Models'] = ['XGBoost                 ']
    
    models = [XGBClassifier(tree_method='gpu_hist', n_estimators= 500, n_jobs = -1)]
        
    scores = []
    
    for i in models:
        model = MultiOutputClassifier(i,n_jobs = -1).fit(X,y)
        probs = model.predict_proba(X)
        val_preds = np.array(probs)[:,:,1].T 
        loss = log_loss(np.ravel(y), np.ravel(val_preds))
        scores.append(loss)

    model_select_df['Scores'] = scores

    
    return model_select_df'''

In [ ]:
test_id = test['sig_id']

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_train

In [ ]:
y_train

In [ ]:
#Hyperparameters from another notebook
#clf = MultiOutputClassifier(XGBClassifier(tree_method='gpu_hist',n_jobs = -1,max_depth=10,max_delta_step= 2.0706,learning_rate= 0.0503,gamma= 3.6975,colsample_bytree= 0.6522,subsample= 0.8639,n_estimators= 800,min_child_weight= 31.5800),n_jobs = -1)

In [ ]:
clf = MultiOutputClassifier(XGBClassifier(tree_method='gpu_hist',n_jobs = -1,),n_jobs = -1)
#clf = MultiOutputClassifier(LGBMClassifier(device='gpu', n_jobs = -1),n_jobs = -1)
#clf = MultiOutputClassifier(CatBoostClassifier(task_type='GPU'),n_jobs = -1)
clf.fit(X_train, y_train)

In [ ]:
X_test = scaler.transform(X_test)

In [ ]:
#pred = clf.predict(X_test)
#pred

probs = clf.predict_proba(X_test)
val_preds = np.array(probs)[:,:,1].T 

pred = val_preds
pred

In [ ]:
pred.sum().sum()

In [ ]:
dfpred = pd.DataFrame(pred, columns = y_train.columns)
dfpred.insert(0, 'sig_id', test_id)
dfpred

In [ ]:
control_mask = test['cp_type']=='ctl_vehicle'

dfpred[control_mask] = 0

dfpred['sig_id'] = test_id

dfpred

In [ ]:
dfpred.to_csv('submission.csv', index = False)